# Canonical task form

Under the hood, all Python, shell and workflow tasks generated by the
`pydra.compose.*.define` decorators/functions are translated to
[dataclass](https://docs.python.org/3/library/dataclasses.html)-like classes by the
[attrs](https://www.attrs.org/en/stable/) library. While the more compact syntax described
in the [Python-tasks](./4-python.html), [Shell-tasks](./5-shell.html) and [Workflow](./6-workflow.html)
tutorials is convenient when designing tasks for specific use cases, it is too magical
for linters follow. Therefore, when designing tasks to be used by third
parties (e.g. `pydra-fsl`, `pydra-ants`) it is recommended to favour the, more
explicit, "canonical" dataclass form.

The syntax of the canonical form is close to that used by the
[Attrs](https://www.attrs.org/en/stable/) package itself, with class type annotations
used to define the fields of the inputs and outputs of the task. Tasks defined in canonical
form will be able to be statically type-checked by [MyPy](https://mypy-lang.org/).

## Python-tasks

Python tasks in dataclass form are decorated by `pydra.compose.python.define`
with inputs listed as type annotations. Outputs are similarly defined in a nested class
called `Outputs`. The function to be executed should be a staticmethod called `function`.
Default values can also be set directly, as with Attrs classes.

In order to allow static type-checkers to check the type of outputs of tasks added
to workflows, it is also necessary to explicitly extend from the `pydra.engine.python.Task`
and `pydra.engine.python.Outputs` classes (they are otherwise set as bases by the
`define` method implicitly). Thus the "canonical form" of Python task is as
follows

In [ ]:
from pydra.utils import print_help
from pydra.compose import python


@python.define
class CanonicalPythonTask(python.Task["CanonicalPythonTask.Outputs"]):
    """Canonical Python task class for testing

    Args:
        a: First input
            to be inputted
        b: Second input
    """

    a: int
    b: float = 2.0  # set default value

    class Outputs(python.Outputs):
        """
        Args:
            c: Sum of a and b
            d: Product of a and b
        """

        c: float
        d: float

    @staticmethod
    def function(a, b):
        return a + b, a / b


print_help(CanonicalPythonTask)

To set additional attributes other than the type and default, such as `allowed_values`
and `validators`, `python.arg` and `python.out` can be used instead.

In [ ]:
import attrs.validators


@python.define
class CanonicalPythonTask(python.Task["CanonicalPythonTask.Outputs"]):
    """Canonical Python task class for testing

    Args:
        a: First input
            to be inputted
        b: Second input
    """

    a: int = python.arg(allowed_values=[1, 2, 3, 4, 5])
    b: float = python.arg(default=2.0, validator=attrs.validators.not_(0))

    class Outputs(python.Outputs):
        """
        Args:
            c: Sum of a and b
            d: Product of a and b
        """

        c: float
        d: float

    @staticmethod
    def function(a, b):
        return a + b, a / b


print_help(CanonicalPythonTask)

## Shell-tasks

The canonical form of shell tasks is the same as for Python tasks, except a string `executable`
attribute replaces the `function` staticmethod.

In [ ]:
import os
from pathlib import Path
from fileformats import generic
from pydra.compose import shell
from pydra.utils.typing import MultiInputObj


@shell.define
class CpWithSize(shell.Task["CpWithSize.Outputs"]):

    executable = "cp"

    in_fs_objects: MultiInputObj[generic.FsObject]
    recursive: bool = shell.arg(argstr="-R")
    text_arg: str = shell.arg(argstr="--text-arg")
    int_arg: int | None = shell.arg(argstr="--int-arg")
    tuple_arg: tuple[int, str] | None = shell.arg(argstr="--tuple-arg")

    class Outputs(shell.Outputs):

        @staticmethod
        def get_file_size(out_file: Path) -> int:
            """Calculate the file size"""
            result = os.stat(out_file)
            return result.st_size

        copied: generic.FsObject = shell.outarg(path_template="copied")
        out_file_size: int = shell.out(callable=get_file_size)


print_help(CpWithSize)

## Workflow definitions

Workflows can also be defined in canonical form, which is the same as for Python tasks
but with a staticmethod called `constructor` that constructs the workflow.

In [ ]:
from pydra.compose import python, workflow
from pydra.utils import print_help, show_workflow


# Example python tasks
@python.define
def Add(a, b):
    return a + b


@python.define
def Mul(a, b):
    return a * b


@workflow.define
class CanonicalWorkflowTask(workflow.Task["CanonicalWorkflowTask.Outputs"]):

    @staticmethod
    def a_converter(value):
        if value is None:
            return value
        return float(value)

    a: int
    b: float = workflow.arg(
        help="A float input",
        converter=a_converter,
    )

    @staticmethod
    def constructor(a, b):
        add = workflow.add(Add(a=a, b=b))
        mul = workflow.add(Mul(a=add.out, b=b))
        return mul.out

    class Outputs(workflow.Outputs):
        out: float


print_help(CanonicalWorkflowTask)
show_workflow(CanonicalWorkflowTask)